In [ ]:
import pandas as pd
import numpy as np
import os
import torch

os.chdir(os.path.join(os.getcwd(), "../code"))

In [21]:

from dataset import *
from plm_base import *
from utils import * 

print(f"[INFO] Initializing PLM with current working directory: {os.getcwd()}")
plm_init(os.path.join(os.getcwd(), "../"))



[INFO] Initializing PLM with current working directory: /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/code


In [6]:
model = plmEmbeddingModel(plm_name="esm2_t33_650M_UR50D", logits_only=True)


In [ ]:
train_test_dataset = PREActivityDataset(
        encoding_identifier="esm2_t33_650M_UR50D",
        evaluation_path="",
        dataset_path="%s/data/gfp_dataset_10mut.csv" % os.path.join(os.getcwd(), "../notebooks"),
        train_indices=None,
        test_indices=None,
        encoding_function=model.encode,
        cache=True,
        lazy_load=True,
        sequence_column_name="full_seq",
        label_column_name="label",
        labels_dtype=torch.float32)

/home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/code/dataset.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.encoded_tensor = torch.load("%s/misc/%s"

[INFO] Caching 
	(1) esm2_t33_650M_UR50D_encoded_sequences.pt


In [20]:
df = pd.read_csv("%s/data/gfp_dataset_10mut.csv" % os.path.join(os.getcwd(), "../notebooks"))


In [36]:

si = np.where(df.columns == "L42")[0][0]
ei = np.where(df.columns == "V224")[0][0]
positions_to_mask = torch.tensor([int(pos[1:]) for pos in df.columns[si:(ei+1)].tolist()], dtype=torch.int)
mask_token = torch.tensor(model.tokenizer.encode("<mask>"), dtype=torch.long)

base_sequence = train_test_dataset.train_dataset.encoded_tensor[0]
base_sequence[positions_to_mask] = mask_token

In [ ]:
logits = model(base_sequence.unsqueeze(0))